<a href="https://colab.research.google.com/github/ken-mao-hk/Ifc-Bounding-Box/blob/main/boundingbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
pip install --upgrade ifcopenshell && pip install --upgrade numpy

In [ ]:
!pip install  --upgrade scipy

In [29]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.shape
import numpy as np


def get_bbox_vertices(bbox_min, bbox_max):
    """Generate all 8 vertices of a bounding box from min/max coordinates"""
    return [
        [bbox_min[0], bbox_min[1], bbox_min[2]],  # 0: min XYZ
        [bbox_max[0], bbox_min[1], bbox_min[2]],  # 1: max X
        [bbox_max[0], bbox_max[1], bbox_min[2]],  # 2: max XY
        [bbox_min[0], bbox_max[1], bbox_min[2]],  # 3: max Y
        [bbox_min[0], bbox_min[1], bbox_max[2]],  # 4: max Z
        [bbox_max[0], bbox_min[1], bbox_max[2]],  # 5: max XZ
        [bbox_max[0], bbox_max[1], bbox_max[2]],  # 6: max XYZ
        [bbox_min[0], bbox_max[1], bbox_max[2]]  # 7: max YZ
    ]


def local_to_global(transform_matrix, local_point):
    """使用形状几何自带的转换矩阵进行坐标转换（修正齐次坐标计算顺序）"""
    local_homogeneous = np.array([*local_point, 1], dtype=np.float64).reshape(-1, 1)  # 转换为列向量
    global_homogeneous = transform_matrix @ local_homogeneous
    global_point = global_homogeneous[:3, 0] / global_homogeneous[3, 0]  # 归一化处理
    return global_point.tolist()


def get_placement_matrix(placement):
    """从IfcLocalPlacement获取转换矩阵（严格右手系构建，修正坐标轴方向定义）"""
    if placement.PlacementRelTo is None:
        parent_matrix = np.eye(4)
    else:
        parent_matrix = get_placement_matrix(placement.PlacementRelTo)

    axis2placement = placement.RelativePlacement
    location = np.array(axis2placement.Location.Coordinates) / 1000.0  # 毫米转米

    rotation_matrix = np.eye(3)
    if hasattr(axis2placement, 'Axis') and hasattr(axis2placement, 'RefDirection'):
        axis = axis2placement.Axis
        ref_dir = axis2placement.RefDirection
        if axis is not None and ref_dir is not None and \
           hasattr(axis, 'DirectionRatios') and hasattr(ref_dir, 'DirectionRatios'):
            axis_vec = np.array(axis.DirectionRatios, dtype=np.float64)
            ref_dir_vec = np.array(ref_dir.DirectionRatios, dtype=np.float64)

            # 标准化向量并构建正交坐标系（修正坐标轴定义顺序）
            axis_vec = axis_vec / np.linalg.norm(axis_vec) if np.linalg.norm(axis_vec) != 0 else np.array([0, 0, 1])
            ref_dir_vec = ref_dir_vec / np.linalg.norm(ref_dir_vec) if np.linalg.norm(ref_dir_vec) != 0 else np.array([1, 0, 0])

            # 定义局部坐标系Z轴为旋转轴（保持与原Axis一致）
            z_axis = axis_vec

            # 定义局部X轴为RefDirection在垂直于Z轴平面上的投影
            x_axis = ref_dir_vec - (ref_dir_vec.dot(z_axis)) * z_axis  # 去除Z轴分量
            x_axis = x_axis / np.linalg.norm(x_axis) if np.linalg.norm(x_axis) != 0 else np.array([1, 0, 0])

            # 定义Y轴为Z×X的右手系方向（确保XY平面与RefDirection一致）
            y_axis = np.cross(z_axis, x_axis)  # 修正叉乘顺序为Z×X得到Y轴
            y_axis = y_axis / np.linalg.norm(y_axis) if np.linalg.norm(y_axis) != 0 else np.array([0, 1, 0])

            rotation_matrix = np.array([x_axis, y_axis, z_axis]).T  # 列向量为局部坐标轴方向

    translation_matrix = np.eye(4)
    translation_matrix[:3, 3] = location

    local_matrix = np.eye(4)
    local_matrix[:3, :3] = rotation_matrix

    combined_matrix = parent_matrix @ translation_matrix @ local_matrix
    return combined_matrix


def get_objects_bounding_boxes(ifc_file_path, ifc_type):
    """计算包含智能尺寸定义的边界框数据（含顶点坐标输出，修正旋转时坐标交换问题）"""
    try:
        ifc_file = ifcopenshell.open(ifc_file_path)
    except Exception as e:
        print(f"Failed to open IFC file: {e}")
        return {}

    settings = ifcopenshell.geom.settings()
    results = {}
    objects = ifc_file.by_type(ifc_type)
    if not objects:
        print(f"No objects of type {ifc_type} found in the IFC file.")
        return results

    for obj in objects:
        try:
            shape = ifcopenshell.geom.create_shape(settings, obj)
            verts = np.array(shape.geometry.verts).reshape(-1, 3)

            # 分离几何变换和对象放置变换
            geometry_transform = np.array(shape.geometry.transformation_matrix, dtype=np.float64) if hasattr(shape.geometry, 'transformation_matrix') else np.eye(4)
            placement_matrix = get_placement_matrix(obj.ObjectPlacement)

            # 组合变换矩阵（正确顺序：先几何变换，再对象放置变换）
            total_transform = placement_matrix @ geometry_transform

            # 获取局部边界框（基于原始几何顶点，不包含任何变换）
            bbox_min, bbox_max = ifcopenshell.util.shape.get_bbox(verts)
            local_vertices = get_bbox_vertices(bbox_min, bbox_max)  # 局部坐标系顶点（几何局部坐标系）

            # 转换为全局坐标系顶点（修正坐标系方向映射）
            global_vertices = [
                local_to_global(total_transform, vertex) for vertex in local_vertices
            ]
            global_vertices_np = np.round(global_vertices, 3).tolist()  # 保留3位小数

            # 尺寸计算（沿用原始逻辑，基于几何局部尺寸）
            local_x = bbox_max[0] - bbox_min[0]
            local_y = bbox_max[1] - bbox_min[1]
            local_z = bbox_max[2] - bbox_min[2]
            horizontal_dims = sorted([local_x, local_y], reverse=True)
            L, W = horizontal_dims[0], horizontal_dims[1]
            H = local_z

            results[obj.GlobalId] = {
                "type": obj.is_a(),
                "dimensions": {
                    "L": np.round(L, 3),
                    "W": np.round(W, 3),
                    "H": np.round(H, 3)
                },
                "coordinates": {
                    "local": [np.round(v, 3).tolist() for v in local_vertices],  # 几何局部坐标顶点
                    "global": global_vertices_np  # 全局坐标顶点
                }
            }

        except Exception as e:
            print(f"Error processing object {obj.GlobalId}: {e}")
            results[obj.GlobalId] = {
                "error": str(e),
                "type": obj.is_a() if hasattr(obj, 'is_a') else "Unknown Type"
            }

    return results


# Example usage
if __name__ == "__main__":
    file_path = "/content/your_model.ifc"
    bboxes = get_objects_bounding_boxes(file_path, "IfcWall")

    if bboxes:
        for guid, data in bboxes.items():
            if "error" in data:
                print(f"\nError occurred for object {guid}: {data['error']}")
                continue

            print(f"\n{data['type']} ({guid}):")
            print(f"Dimensions (LxWxH): {data['dimensions']['L']} x {data['dimensions']['W']} x {data['dimensions']['H']}")

            print("\nLocal Coordinates (Bounding Box Vertices):")
            for i, vertex in enumerate(data['coordinates']['local']):
                print(f"Vertex {i}: {vertex}")

            print("\nGlobal Coordinates (Bounding Box Vertices):")
            for i, vertex in enumerate(data['coordinates']['global']):
                print(f"Vertex {i}: {vertex}")

    else:
        print("No valid bounding box data was obtained.")



IfcWallStandardCase (072ZP5RUb3gBTy6zAYFpDy):
Dimensions (LxWxH): 5.305 x 0.29 x 4.5

Local Coordinates (Bounding Box Vertices):
Vertex 0: [0.0, -0.145, 0.0]
Vertex 1: [5.305, -0.145, 0.0]
Vertex 2: [5.305, 0.145, 0.0]
Vertex 3: [0.0, 0.145, 0.0]
Vertex 4: [0.0, -0.145, 4.5]
Vertex 5: [5.305, -0.145, 4.5]
Vertex 6: [5.305, 0.145, 4.5]
Vertex 7: [0.0, 0.145, 4.5]

Global Coordinates (Bounding Box Vertices):
Vertex 0: [-3.087, 0.887, 0.0]
Vertex 1: [2.218, 0.887, 0.0]
Vertex 2: [2.218, 1.177, 0.0]
Vertex 3: [-3.087, 1.177, 0.0]
Vertex 4: [-3.087, 0.887, 4.5]
Vertex 5: [2.218, 0.887, 4.5]
Vertex 6: [2.218, 1.177, 4.5]
Vertex 7: [-3.087, 1.177, 4.5]

IfcWallStandardCase (1kI6jlbn5A0gu2yzuvR6iD):
Dimensions (LxWxH): 7.083 x 0.29 x 4.5

Local Coordinates (Bounding Box Vertices):
Vertex 0: [0.0, -0.145, 0.0]
Vertex 1: [7.083, -0.145, 0.0]
Vertex 2: [7.083, 0.145, 0.0]
Vertex 3: [0.0, 0.145, 0.0]
Vertex 4: [0.0, -0.145, 4.5]
Vertex 5: [7.083, -0.145, 4.5]
Vertex 6: [7.083, 0.145, 4.5]
Vertex 